# hackathon

In [2]:
import pandas as pd
import numpy as np

In [3]:
airline_df = pd.read_csv('/content/airline.csv')
print("Airline Data:")
airline_df.head()
# print(airline_df.columns)
# airline_df.shape

Airline Data:


,airline_name,link,title,author,author_country,date,content,aircraft,type_traveller,cabin_flown,route,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,wifi_connectivity_rating,value_money_rating,recommended
0,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,D Ito,Germany,2015-04-10,Outbound flight FRA/PRN A319. 2 hours 10 min f...,NaN,NaN,Economy,NaN,7.0,4.0,4.0,4.0,0.0,NaN,NaN,4.0,1
1,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,Ron Kuhlmann,United States,2015-01-05,Two short hops ZRH-LJU and LJU-VIE. Very fast ...,NaN,NaN,Business Class,NaN,10.0,4.0,5.0,4.0,1.0,NaN,NaN,5.0,1
2,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,E Albin,Switzerland,2014-09-14,Flew Zurich-Ljubljana on JP365 newish CRJ900. ...,NaN,NaN,Economy,NaN,9.0,5.0,5.0,4.0,0.0,NaN,NaN,5.0,1
3,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,Tercon Bojan,Singapore,2014-09-06,Adria serves this 100 min flight from Ljubljan...,NaN,NaN,Business Class,NaN,8.0,4.0,4.0,3.0,1.0,NaN,NaN,4.0,1
4,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,L James,Poland,2014-06-16,WAW-SKJ Economy. No free snacks or drinks on t...,NaN,NaN,Economy,NaN,4.0,4.0,2.0,1.0,2.0,NaN,NaN,2.0,0


In this code i have selected the statment form content column and then filtered them to get independent words. then calculted probability of all p(w1/r), p(w2/r) , p(w3/r)...p(wn/r).
And then stored all the probabilities of all the probability value of each word with each rating into dictionary, for all the rating types


In [4]:
import nltk
from nltk.corpus import stopwords
import string
import pickle


nltk.download('stopwords')

rating_columns = ['overall_rating', 'seat_comfort_rating', 'cabin_staff_rating',
                  'food_beverages_rating', 'inflight_entertainment_rating',
                  'value_money_rating']

stop_words = set(stopwords.words('english'))

def preprocess_text(text):

    text = text.translate(str.maketrans('', '', string.punctuation))

    words = text.lower().split()
    words = [word for word in words if word not in stop_words]
    return words

p_r = {}
p_w_given_r = {}
word_counts_per_rating = {}
total_word_counts_per_rating = {}
vocabulary = set()

total_documents = 0
rating_counts = {}

for rating_column in rating_columns:
    print(f"Processing rating category: {rating_column}")

    rating_counts = {}
    word_counts_per_rating = {}
    total_word_counts_per_rating = {}
    vocabulary = set()
    total_documents = 0

    for index, row in airline_df.iterrows():
        content = str(row['content'])
        words = preprocess_text(content)

        try:
            rating_value = int(row[rating_column])
        except (ValueError, TypeError):
            continue

        total_documents += 1
        rating_counts[rating_value] = rating_counts.get(rating_value, 0) + 1
        word_counts = word_counts_per_rating.get(rating_value, {})
        total_word_counts_per_rating[rating_value] = total_word_counts_per_rating.get(rating_value, 0) + len(words)

        for word in words:
            vocabulary.add(word)
            word_counts[word] = word_counts.get(word, 0) + 1

        word_counts_per_rating[rating_value] = word_counts

    p_r[rating_column] = {}
    for rating in rating_counts:
        p_r[rating_column][rating] = rating_counts[rating] / total_documents

    p_w_given_r[rating_column] = {}
    for rating in rating_counts:
        word_counts = word_counts_per_rating[rating]
        total_words = total_word_counts_per_rating[rating]
        p_w_given_r[rating_column][rating] = {}
        for word in vocabulary:

            word_count = word_counts.get(word, 0) + 1
            total_words_smooth = total_words + len(vocabulary)
            p_w_given_r[rating_column][rating][word] = word_count / total_words_smooth

    with open(f'p_r_{rating_column}.pkl', 'wb') as f:
        pickle.dump(p_r[rating_column], f)
    with open(f'p_w_given_r_{rating_column}.pkl', 'wb') as f:
        pickle.dump(p_w_given_r[rating_column], f)
    with open(f'vocabulary_{rating_column}.pkl', 'wb') as f:
        pickle.dump(vocabulary, f)

    print(f"Completed processing for {rating_column}")

print("Training phase completed.")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Processing rating category: overall_rating
Completed processing for overall_rating
Processing rating category: seat_comfort_rating
Completed processing for seat_comfort_rating
Processing rating category: cabin_staff_rating
Completed processing for cabin_staff_rating
Processing rating category: food_beverages_rating
Completed processing for food_beverages_rating
Processing rating category: inflight_entertainment_rating
Completed processing for inflight_entertainment_rating
Processing rating category: value_money_rating
Completed processing for value_money_rating
Training phase completed.


# Prediction

used p(r/w1...w2) = p(w1/r)p(w2/r)....p(wn/r)/p(w1)p(w2)...p(wn).
the answer will be the rating with highest probability using argmax.

In [19]:
import numpy as np
import pickle
import nltk
from nltk.corpus import stopwords
import string

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))

    words = text.lower().split()
    words = [word for word in words if word not in stop_words]
    return words

def calculate_rating_for_category(user_input, rating_column):

    with open(f'p_r_{rating_column}.pkl', 'rb') as f:
        p_r_category = pickle.load(f)
    with open(f'p_w_given_r_{rating_column}.pkl', 'rb') as f:
        p_w_given_r_category = pickle.load(f)
    with open(f'vocabulary_{rating_column}.pkl', 'rb') as f:
        vocabulary = pickle.load(f)

    words = preprocess_text(user_input)

    log_probabilities = {}

    for rating in p_r_category.keys():

        log_prob = np.log(p_r_category[rating])

        for word in words:

            if word in vocabulary:

                p_w_given_r_value = p_w_given_r_category[rating].get(word, 1 / (len(vocabulary)))
            else:

                p_w_given_r_value = 1 / (len(vocabulary))

            log_prob += np.log(p_w_given_r_value)

        log_probabilities[rating] = log_prob

    predicted_rating = max(log_probabilities, key=log_probabilities.get)

    return predicted_rating

user_input = input("Please enter your review or statement about the flight: ")

predicted_ratings = {}
for rating_column in rating_columns:
    predicted_ratings[rating_column] = calculate_rating_for_category(user_input, rating_column)

for category, rating in predicted_ratings.items():
    print(f"Predicted {category.replace('_', ' ').title()}: {rating}")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


KeyboardInterrupt: Interrupted by user